# Code-to-Code Comparison: Dinwoodie

### National Renewable Energy Laboratory

#### Rob Hammond

##### 29 March 2022

In [1]:
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core.library import DINWOODIE, load_yaml

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [2]:
# Converting Labor values to fixed cost input for the base case
tech_salary_annual = 80000
techs = 20
capacity = 240 * 1000  # 240 -> kW
f"{tech_salary_annual * techs / capacity:.4f}"

'6.6667'

In [3]:
configs = [
    "base",
    "more_ctvs",
    "fewer_ctvs",
    "more_techs",
    "fewer_techs",
    "failure_50",
    "failure_200",
    "no_hlvs",
    "no_weather",
    "historic_weather",
    "manual_resets_only",
    "minor_repairs_only",
    "medium_repairs_only",
    "major_repairs_only",
    "major_replacements_only",
    "annual_service_only",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "fsv cost": [],
    "hlv cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
    "ctv utilization": [],
    "fsv utilization": [],
    "hlv utilization": [],
    
}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(DINWOODIE, f"{config}.yaml")
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(30)} | {(end - start) / 60:.2f} m")
    
    # Gather the results of interest
    years = sim.metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability_time = sim.metrics.time_based_availability(frequency="project", by="windfarm")
    availability_production = sim.metrics.production_based_availability(frequency="project", by="windfarm")
    cf_net = sim.metrics.capacity_factor(which="net", frequency="project", by="windfarm")
    cf_gross = sim.metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
    power_production = sim.metrics.power_production(frequency="project", by_turbine=False).values[0][0]
    completion_rate = sim.metrics.task_completion_rate(which="both", frequency="project")
    parts = sim.metrics.events[["materials_cost"]].sum().sum()
    techs = sim.metrics.project_fixed_costs(frequency="project", resolution="low").operations[0]
    total = sim.metrics.events[["total_cost"]].sum().sum()
    
    equipment = sim.metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Heavy Lift Vessel" in el]].sum().sum()
    fsv = equipment[[el for el in equipment.columns if "Field Support Vessel" in el]].sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    
    utilization = sim.metrics.service_equipment_utilization(frequency="project")
    hlv_ur = utilization[[el for el in utilization.columns if "Heavy Lift Vessel" in el]].mean().mean()
    fsv_ur = utilization[[el for el in utilization.columns if "Field Support Vessel" in el]].mean().mean()
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    
    # Log the results of interest
    results["availability - time based"].append(availability_time)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["annual direct O&M cost"].append((total + techs) / mil / years)
    results["annual vessel cost"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["fsv cost"].append(fsv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["annual repair cost"].append(parts / mil / years)
    results["annual technician cost"].append(techs / mil / years)
    results["ctv utilization"].append(ctv_ur)
    results["fsv utilization"].append(fsv_ur)
    results["hlv utilization"].append(hlv_ur)
    sim.env.cleanup_log_files(log_only=True)

                dinwoodie_base | 2.53 m
           dinwoodie_more_ctvs | 2.65 m
          dinwoodie_fewer_ctvs | 2.76 m
          dinwoodie_more_techs | 2.88 m
         dinwoodie_fewer_techs | 2.90 m
          dinwoodie_failure_50 | 2.45 m
         dinwoodie_failure_200 | 3.96 m
             dinwoodie_no_hlvs | 3.34 m
          dinwoodie_no_weather | 3.02 m
    dinwoodie_historic_weather | 2.64 m
  dinwoodie_manual_resets_only | 2.46 m
  dinwoodie_minor_repairs_only | 2.03 m
 dinwoodie_medium_repairs_only | 1.80 m
  dinwoodie_major_repairs_only | 1.91 m
dinwoodie_major_replacements_only | 1.84 m
 dinwoodie_annual_service_only | 2.57 m


In [5]:
# Save the results
# pickled dictionary format
with open(DINWOODIE / "outputs" / "results_dict_v0.4.1.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(DINWOODIE / "outputs" / "results_data_v0.4.1.csv", index_label="result")

In [6]:
results_df

,base,more_ctvs,fewer_ctvs,more_techs,fewer_techs,failure_50,failure_200,no_hlvs,no_weather,historic_weather,manual_resets_only,minor_repairs_only,medium_repairs_only,major_repairs_only,major_replacements_only,annual_service_only
availability - time based,0.96,0.96,0.97,0.96,0.96,0.96,0.96,0.98,0.97,0.97,1.00,1.00,1.00,1.00,0.97,0.98
availability - production based,0.96,0.96,0.97,0.96,0.96,0.96,0.96,0.98,0.00,0.97,1.00,1.00,1.00,1.00,0.97,0.98
capacity factor - net,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.00,0.46,0.47,0.47,0.47,0.47,0.46,0.47
capacity factor - gross,0.48,0.47,0.47,0.47,0.47,0.47,0.47,0.47,0.00,0.48,0.47,0.47,0.47,0.47,0.47,0.47
power production,"9,663,286,868.00","9,184,744,191.50","9,242,678,298.50","9,236,517,517.00","9,216,369,008.50","9,224,938,932.50","9,219,047,102.50","9,368,721,519.50",0.00,"7,807,615,395.00","9,542,484,300.00","9,541,607,311.50","9,559,704,061.50","9,572,313,662.50","9,293,465,241.00","9,422,005,405.00"
task completion rate,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.86,0.97,1.00
annual direct O&M cost,18.46,23.17,15.43,16.90,17.21,10.15,24.54,5.71,17.58,15.21,3.28,3.65,3.85,3.74,15.02,4.73
annual vessel cost,13.10,15.27,10.36,11.17,12.67,6.11,17.46,1.85,12.20,10.30,1.75,1.83,1.84,1.99,11.38,1.79
ctv cost,1.92,3.12,1.82,1.86,1.84,1.83,1.89,1.85,1.86,1.83,1.75,1.83,1.84,1.84,1.91,1.79
fsv cost,0.15,0.17,0.15,0.11,0.13,0.07,0.40,0.00,0.13,0.19,0.00,0.00,0.00,0.15,0.00,0.00
